# █████████████████████████
# H4S2-Model-Hyperparameter-Search
This repository was created in association with the [paper citation needed]
# █████████████████████████

This Interactive Jupyter Notebook was created in association with the [paper citation needed]

This Notebook performs a final exploration of the selected hyperparameter for more the H4S2 model. Keeping all other hyperparamers constant from previous optimization steps, we varied the:
- Dropout
- Learning Rate
- Hidden layer's Activation Function
- Output layer's Activation Function
- Hidden layer's architecture
- Hidden layer's use of bias

saving the results of each hyperparameter search as a excel file in the current directory

# ███████████████████
# Setting Current Directory
## The Location of Files being Used
# ███████████████████

In [2]:
cd C:\Users\jchar\Documents\Research\Rodger's lab\Research Results\NN RSS Prediction Model\Model 9

C:\Users\jchar\Documents\Research\Rodger's lab\Research Results\NN RSS Prediction Model\Model 9


In [ ]:
#cd C:\Users\File\Path\To\DatasetFiles\StorageFolder

In [4]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 2863-468B

 Directory of C:\Users\jchar\Documents\Research\Rodger's lab\Research Results\NN RSS Prediction Model\Model 9

07/31/2024  04:55 PM    <DIR>          .
06/04/2024  01:40 PM    <DIR>          ..
09/19/2023  02:50 PM             5,090 12RSS_fasta_RSSsite.txt
09/13/2023  06:20 PM            11,608 12RSS_RIC_script.pl
06/04/2024  01:42 PM    <DIR>          Files to start colab figure notebook
07/25/2024  09:55 AM           788,858 H4S2 NN paper's Pearson r Correlations and figure 3 and 5.docx
07/19/2024  11:21 AM         1,180,747 H4S2_predictions_output.xlsx
06/08/2024  04:24 PM            14,540 hyperparameter_search_hidden_activation_function.xlsx
06/09/2024  08:25 AM             8,755 hyperparameter_search_hidden_bias.xlsx
06/08/2024  05:38 PM            32,903 hyperparameter_search_hidden_layer_architecture.xlsx
06/08/2024  01:29 PM            17,073 hyperparameter_search_lrate.xlsx
06/08/2024  04:53 PM            15

# ███████████████████
# Importing Required Packages
# ███████████████████

In [10]:
import pandas as pd  # pandas Version 1.5.3
import numpy as np  # numpy Version 1.24.3
import os
import itertools
import tensorflow as tf  # tensorflow Version 2.15.0
from tensorflow import keras  # keras Version 2.15.0
from keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_addons as tfa  # tensorflow_addons Version 0.22.0
from sklearn.model_selection import StratifiedKFold  # scikit-learn Version 1.3.0
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt  # matplotlib Version 3.8.3
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from scipy import stats  # scipy Version 1.10.1
from scipy.stats import ks_2samp, spearmanr

In [26]:
import importlib.metadata

# List of package names you want to check
packages = [
    'pandas',
    'numpy',
    'itertools',
    'tensorflow',
    'keras', 
    'tensorflow_addons',
    'scikit-learn',
    'matplotlib',
    'scipy',
]

# Print the version of each package
for package in packages:
    try:
        version = importlib.metadata.version(package)
        print(f"{package}: {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{package} is not installed.")


pandas: 1.5.3
numpy: 1.24.3
itertools is not installed.
tensorflow: 2.15.0
keras: 2.15.0
tensorflow_addons: 0.22.0
scikit-learn: 1.3.0
matplotlib: 3.8.3
scipy: 1.10.1
importlib is not installed.


# ███████████████████
# Setting Dataset Files into a Dictionary
## callable datasets, easy to itterate through
# ███████████████████

In [16]:
# Set up dictionary to store file paths and flanking sequences to be added later
datasets = {
    "iseq1": {"file": "SJ-0 SARP-seq H4-S2 iSeq1.tabular", "additional_sequence": [0, "ACAGACTGGAACAAAAACC"]},
    "iseq2": {"file": "SJ-0 SARP-seq H4-S2 iSeq2.tabular", "additional_sequence": [0, "ACAGACTGGAACAAAAACC"]},
    "miseq1": {"file": "SJ-0 SARP-seq H4-S2 miSeq1.tabular", "additional_sequence": [0, "ACAGACTGGAACAAAAACC"]},
    "pax3_lmo2_crypticnonamer": {"file": "SARP_seq_H4-7S2_cNon_wholeRSS.txt", "additional_sequence": [0, 0]},
    "cf1_crypticnonamer": {"file": "SARP_seq_H4-7S2_cNon_CF1_v2.txt", "additional_sequence": [0, 0]}
}

# Convert the dictionary to a Pandas DataFrame
datasets_df = pd.DataFrame.from_dict(datasets, orient='index')

# Display the table
print(datasets_df)

                                                        file  \
iseq1                      SJ-0 SARP-seq H4-S2 iSeq1.tabular   
iseq2                      SJ-0 SARP-seq H4-S2 iSeq2.tabular   
miseq1                    SJ-0 SARP-seq H4-S2 miSeq1.tabular   
pax3_lmo2_crypticnonamer   SARP_seq_H4-7S2_cNon_wholeRSS.txt   
cf1_crypticnonamer           SARP_seq_H4-7S2_cNon_CF1_v2.txt   

                               additional_sequence  
iseq1                     [0, ACAGACTGGAACAAAAACC]  
iseq2                     [0, ACAGACTGGAACAAAAACC]  
miseq1                    [0, ACAGACTGGAACAAAAACC]  
pax3_lmo2_crypticnonamer                    [0, 0]  
cf1_crypticnonamer                          [0, 0]  


## • Pulling data into dictionary of dataframes

In [17]:
def text_file_to_dataframe(file, delimiter='\t', additional_sequence=[0, 0]):
    """
    Reads a text file into a DataFrame, standardizes column names, and appends sequences as needed.

    Parameters:
    - file (str): The path to the input text file.
    - delimiter (str, optional): The delimiter used in the text file. Default is '\t' (tab).
    - additional_sequence (list of str, optional): A list containing two elements, representing upstream and downstream sequences.
      Default is [0, 0], indicating no additional sequences.

    Returns:
    - pd.DataFrame: A DataFrame containing the processed data.

    Raises:
    - ValueError: If the input file does not have exactly two columns, or if there is an issue with recombination | sequence data columns.
    """

    # Appends flanking sequence with additional sequence
    def append_sequence(sequence, additional_sequence):
        """
        Appends DNA sequences based by adding upstream and downstream elements.

        Parameters:
        - sequence (str): The original DNA sequence.
        - additional_sequence (list of str): A list containing two elements, representing upstream and downstream sequences.

        Returns:
        - str: The modified DNA sequence after appending the additional sequences.
        """
        # Separate appending sequences
        upstream, downstream = map(str, additional_sequence)
        # Append sequence
        if upstream == '0':
            return sequence + downstream
        elif downstream == '0':
            return upstream + sequence
        else:
            return upstream + sequence + downstream

    # Read the text file into a DataFrame
    with open(file, 'r') as f:
        first_line = f.readline().strip()
        has_header = 'Sequence' in first_line or 'Readcount' in first_line or 'Read count' in first_line

    if has_header:
        working_df = pd.read_csv(file, delimiter=delimiter)
    else:
        working_df = pd.read_csv(file, delimiter=delimiter, header=None)

    # Check for the number of columns
    if len(working_df.columns) != 2:
        raise ValueError("The input file should have exactly two columns. [Recombination data | DNA sequence]")

    # Identify numeric columns
    recombination_column = [column for column in working_df.columns if pd.to_numeric(working_df[column], errors='coerce').notna().any()]

    if len(recombination_column) == 0:
        raise ValueError("No recombination data column found. [Recombination data | DNA sequence]")
    elif len(recombination_column) == 2:
        raise ValueError("Both columns are numeric. Please provide one column with DNA sequence. [Recombination data | DNA sequence]")

    # Assume the sequence column is the DNA sequence column
    sequence_column = [column for column in working_df.columns if column not in recombination_column][0]

    # Apply the sequence appending function
    if additional_sequence != [0,0]:
        working_df[sequence_column] = working_df.apply(lambda row: append_sequence(row[sequence_column], additional_sequence), axis=1)

    # Rename columns
    working_df.rename(columns={recombination_column[0]: 'Read count', sequence_column: 'Sequence'}, inplace=True)

    return working_df

# Create an empty dictionary to store DataFrames
result_dataframes = {}

# Iterate through datasets
for index, row in datasets_df.iterrows():
    try:
        df = text_file_to_dataframe(row['file'], additional_sequence=row['additional_sequence'])
        print(f"Dataset: {index}\n")
        print(df.head(2))

        # Store the DataFrame in the dictionary using the dataset index as the key
        result_dataframes[index] = df
    except ValueError as e:
        print(f"Error processing dataset {index}: {e}\n")

Dataset: iseq1

   Read count                      Sequence
0       18982  CACAGTGATACAGACTGGAACAAAAACC
1       15235  CACTATGATACAGACTGGAACAAAAACC
Dataset: iseq2

   Read count                      Sequence
0         961  CACAGTGCTACAGACTGGAACAAAAACC
1         960  CACTGTGTTACAGACTGGAACAAAAACC
Dataset: miseq1

   Read count                      Sequence
0       26810  CACAGTGCTACAGACTGGAACAAAAACC
1       24599  CACTATGATACAGACTGGAACAAAAACC
Dataset: pax3_lmo2_crypticnonamer

   Read count                      Sequence
0        5177  CACGGTGATACAGACTGGACTAAAAACC
1        4709  CACAGTGATACAGACTGGACTAAAAACC
Dataset: cf1_crypticnonamer

    Read count                      Sequence
0  8632.666667  CACTATGATACAGACTGGAACAAAAACC
1  8458.666667  CACTGTGATACAGACTGGAACAAAAACC


## • Separating cryptic nonamer datasets for processing

In [18]:
# Split pax3_lmo2_crypticnonamer into lmo2_crypticnonamer and pax3_crypticnonamer
pax3_lmo2_crypticnonamer_df = result_dataframes['pax3_lmo2_crypticnonamer']

# Define the criteria for splitting based on the last 9 nucleotides
pax3_criteria = pax3_lmo2_crypticnonamer_df['Sequence'].str.endswith('CTAAAAACC')
lmo2_criteria = pax3_lmo2_crypticnonamer_df['Sequence'].str.endswith('TGGAAAATA')

# Apply the criteria to split the DataFrame
pax3_crypticnonamer_df = pax3_lmo2_crypticnonamer_df[pax3_criteria].reset_index(drop=True)
lmo2_crypticnonamer_df = pax3_lmo2_crypticnonamer_df[lmo2_criteria].reset_index(drop=True)

# Add the split datasets to the result_dataframes dictionary
result_dataframes["pax3_crypticnonamer"] = pax3_crypticnonamer_df
result_dataframes["lmo2_crypticnonamer"] = lmo2_crypticnonamer_df

# Remove the original pax3_lmo2_crypticnonamer dataset from the dictionary
del result_dataframes['pax3_lmo2_crypticnonamer']

# Check the resulting dataframes
for dataset_name, dataset_df in result_dataframes.items():
    print(f"\nDataset: {dataset_name}\n")
    print(dataset_df.head(2))


Dataset: iseq1

   Read count                      Sequence
0       18982  CACAGTGATACAGACTGGAACAAAAACC
1       15235  CACTATGATACAGACTGGAACAAAAACC

Dataset: iseq2

   Read count                      Sequence
0         961  CACAGTGCTACAGACTGGAACAAAAACC
1         960  CACTGTGTTACAGACTGGAACAAAAACC

Dataset: miseq1

   Read count                      Sequence
0       26810  CACAGTGCTACAGACTGGAACAAAAACC
1       24599  CACTATGATACAGACTGGAACAAAAACC

Dataset: cf1_crypticnonamer

    Read count                      Sequence
0  8632.666667  CACTATGATACAGACTGGAACAAAAACC
1  8458.666667  CACTGTGATACAGACTGGAACAAAAACC

Dataset: pax3_crypticnonamer

   Read count                      Sequence
0        5177  CACGGTGATACAGACTGGACTAAAAACC
1        4709  CACAGTGATACAGACTGGACTAAAAACC

Dataset: lmo2_crypticnonamer

   Read count                      Sequence
0         499  CACAGTGATACAGACTGGATGGAAAATA
1         460  CACGGTGATACAGACTGGATGGAAAATA


# ██████████████
# Dataset Processing
# ██████████████


## • Adding zero scoring sequences to datasets

In [19]:
def fill_zero_scoring_samples(output_dataset, degenerate_motif, starting_position):
    """
    Fills in the zero-scoring samples into the output dataset based on the degenerate motif.

    Parameters:
    - output_dataset (pd.DataFrame): The output dataset containing read counts and sequences.
    - degenerate_motif (str): The degenerate motif representing the input dataset.
    - starting_position (int): The position where the degenerate motif starts.

    Returns:
    - pd.DataFrame: The updated output dataset with filled zero-scoring samples.
    """
    # Define the mapping of IUPAC nucleotide codes to actual nucleotides
    nucleotide_mapping = {
        'A': 'A',
        'T': 'T',
        'G': 'G',
        'C': 'C',
        'N': 'ATGC',
        'K': 'GT'
    }

    # Generate all possible sequences by replacing each degenerate nucleotide with its mapping
    all_sequences = [''.join(combination) for combination in itertools.product(*[nucleotide_mapping[nuc] for nuc in degenerate_motif])]

    # Infer ending position based on starting position and degenerate motif length
    ending_position = starting_position + len(degenerate_motif)

    # Extract flanking sequences from the output dataset
    starting_sequence = output_dataset['Sequence'].str.slice(0, starting_position).unique()[0]
    ending_sequence = output_dataset['Sequence'].str.slice(ending_position).unique()[0]

    # Validate starting and ending sequences
    if starting_sequence is None or ending_sequence is None:
        raise ValueError("Starting or ending sequences are not consistent across all samples.")

    # Generate all possible sequences based on the degenerate motif and flanking sequences
    generated_sequences = [starting_sequence + sequence + ending_sequence for sequence in all_sequences]

    # Merge generated sequences with the output dataset, filling in zero scores where necessary
    filled_dataset = pd.merge(output_dataset, pd.DataFrame({'Sequence': generated_sequences}), on='Sequence', how='right')
    filled_dataset['Read count'].fillna(0, inplace=True)

    return filled_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["iseq1"], degenerate_motif = 'NNNNNN', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["iseq1"] = filled_output_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["iseq2"], degenerate_motif = 'NNNNNN', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["iseq2"] = filled_output_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["miseq1"], degenerate_motif = 'NNNNNN', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["miseq1"] = filled_output_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["cf1_crypticnonamer"], degenerate_motif = 'NNNNAK', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["cf1_crypticnonamer"] = filled_output_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["pax3_crypticnonamer"], degenerate_motif = 'NNNNAK', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["pax3_crypticnonamer"] = filled_output_dataset

# Add zero scoring sequences to the dataset
filled_output_dataset = fill_zero_scoring_samples(result_dataframes["lmo2_crypticnonamer"], degenerate_motif = 'NNNNAK', starting_position = 3)
# Update result_dataframes with the filled output dataset
result_dataframes["lmo2_crypticnonamer"] = filled_output_dataset

In [20]:
# Check the resulting dataframes
for dataset_name, dataset_df in result_dataframes.items():
    print(f"\nDataset: {dataset_name}\n")
    print(dataset_df.head(2))
    print(dataset_df.shape)    


Dataset: iseq1

   Read count                      Sequence
0         2.0  CACAAAAAAACAGACTGGAACAAAAACC
1        12.0  CACAAAAATACAGACTGGAACAAAAACC
(4096, 2)

Dataset: iseq2

   Read count                      Sequence
0         0.0  CACAAAAAAACAGACTGGAACAAAAACC
1         0.0  CACAAAAATACAGACTGGAACAAAAACC
(4096, 2)

Dataset: miseq1

   Read count                      Sequence
0         7.0  CACAAAAAAACAGACTGGAACAAAAACC
1       262.0  CACAAAAATACAGACTGGAACAAAAACC
(4096, 2)

Dataset: cf1_crypticnonamer

   Read count                      Sequence
0    8.666667  CACAAAAAGACAGACTGGAACAAAAACC
1    5.000000  CACAAAAATACAGACTGGAACAAAAACC
(512, 2)

Dataset: pax3_crypticnonamer

   Read count                      Sequence
0        38.0  CACAAAAAGACAGACTGGACTAAAAACC
1        26.0  CACAAAAATACAGACTGGACTAAAAACC
(512, 2)

Dataset: lmo2_crypticnonamer

   Read count                      Sequence
0         0.0  CACAAAAAGACAGACTGGATGGAAAATA
1         0.0  CACAAAAATACAGACTGGATGGAAAATA
(512, 2)


## • Pull H4S2, N1N3, N8N9 into new columns

In [21]:
def extract_and_add_column(dataframes, start_position, end_position, column_name):
    """
    Extracts sequences from specified positions and adds a new column to each DataFrame.

    Parameters:
    - dataframes: Dictionary of DataFrames where keys are dataset names.
    - start_position: Starting position for sequence extraction.
    - end_position: Ending position for sequence extraction.
    - column_name: Name to be assigned to the new DataFrame column.

    Returns:
    - None (modifies input dictionaries in-place).
    """
    for dataset_name, dataset_df in dataframes.items():
        # Extract the 'Sequence' column
        sequences = dataset_df['Sequence']

        # Split each sequence into the specified positions
        extracted_sequence = sequences.str.slice(start_position, end_position + 1)

        # Add the new column to the DataFrame
        dataframes[dataset_name][column_name] = extracted_sequence

#splitting sequence into partial coverage
extract_and_add_column(result_dataframes, 3, 8, 'H4S2')
extract_and_add_column(result_dataframes, 19, 21, 'N1N3')
extract_and_add_column(result_dataframes, 26, 28, 'N8N9')


# Display the modified DataFrames
for dataset_name, dataset_df in result_dataframes.items():
    print(f"Dataset: {dataset_name}\n")
    print(dataset_df)

Dataset: iseq1

      Read count                      Sequence    H4S2 N1N3 N8N9
0            2.0  CACAAAAAAACAGACTGGAACAAAAACC  AAAAAA  ACA   CC
1           12.0  CACAAAAATACAGACTGGAACAAAAACC  AAAAAT  ACA   CC
2           53.0  CACAAAAAGACAGACTGGAACAAAAACC  AAAAAG  ACA   CC
3            2.0  CACAAAAACACAGACTGGAACAAAAACC  AAAAAC  ACA   CC
4            3.0  CACAAAATAACAGACTGGAACAAAAACC  AAAATA  ACA   CC
...          ...                           ...     ...  ...  ...
4091         0.0  CACCCCCGCACAGACTGGAACAAAAACC  CCCCGC  ACA   CC
4092         1.0  CACCCCCCAACAGACTGGAACAAAAACC  CCCCCA  ACA   CC
4093         0.0  CACCCCCCTACAGACTGGAACAAAAACC  CCCCCT  ACA   CC
4094         0.0  CACCCCCCGACAGACTGGAACAAAAACC  CCCCCG  ACA   CC
4095         0.0  CACCCCCCCACAGACTGGAACAAAAACC  CCCCCC  ACA   CC

[4096 rows x 5 columns]
Dataset: iseq2

      Read count                      Sequence    H4S2 N1N3 N8N9
0            0.0  CACAAAAAAACAGACTGGAACAAAAACC  AAAAAA  ACA   CC
1            0.0  CACAAAAATACAGAC

# ███████████████████████████████████████
# Establishing Functions for stratified k-fold cross validation
# ███████████████████████████████████████

In [22]:
def FVAF(real_label,predicted_label):
    """
    Calculates the Fractional Variance Accounted For (FVAF) between real and predicted values.

    Args:
        real_label (np.ndarray): The array of real values (ground truth).
        predicted_label (np.ndarray): The array of predicted values.

    Returns:
        float: The Fraction of Variance Accounted For (FVAF) metric.
    """
    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(real_label, predicted_label)

    # Calculate the variance (VAR)
    var = np.var(real_label)

    # Calculate 1-MSE/VAR
    result = 1 - mse/var
    
    return result

def calculate_spearman_correlation(actual, predicted):
    # Calculate Spearman correlation coefficient
    correlation, _ = spearmanr(actual, predicted)
    return correlation

def calculate_rmse(actual, predicted):
    # Calculate Root Mean Squared Error (RMSE)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    return rmse

def plot_bins_read_count_by_rank(working_df, bin_ranges, read_count_column_name = 'Read count', plt_log_transform = False):
    """
    Plot the bins read count by rank.
    
    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the Read counts and bin assignments.
        bin_ranges (pandas.DataFrame): Input Dataframe containing read counts and bin assignments.
        read_count_column_name (str): Name of the column in the text file that contains the Read Count. Defaults to 'Read count'.
        plt_log_transform (bool):
            -False (default): Plots read count as in working_df
            -True: Plots a log 10 transform of read counts.
    
    Returns:
        None
    
    """
    # Set color values for bins
    colors = [(0.3882, 0.5804, 0.7765),  # medium blue
          (0.6902, 0.7686, 0.8706),  # light blue
          (0.0667, 0.3020, 0.5490),  # dark blue
          (0.9020, 0.6706, 0.0078),  # orange
          (0.6510, 0.4627, 0.1137),  # brown
          (0.5176, 0.2353, 0.2235),  # maroon
          (0.8471, 0.6039, 0.8549),  # lavender
          (0.5490, 0.1647, 0.6510),  # purple
          (0.4078, 0.4078, 0.4078)]  # dark gray
    cmap = mcolors.ListedColormap(colors)
    
    # Check if the read_count_column_name exists in the DataFrame
    if read_count_column_name not in working_df.columns:
        raise ValueError(f"The specified read_count_column_name '{read_count_column_name}' does not exist in the file.\nThe file contains: '{list(working_df.columns)}'")
    
    # Sort the DataFrame by Read count in descending order
    sorted_working_df = working_df.sort_values(by=read_count_column_name, ascending=False)

    # Define the legend handles and labels based on bin_range
    legend_handles = []
    legend_labels = []
    for i in range(len(bin_ranges)):
        bin_label = 'Bin {}:{}'.format(bin_ranges.iloc[i]['Bin#'], bin_ranges.iloc[i]['Bin Read Count Range'])
        bin_patch = mpatches.Patch(color=colors[i], label=bin_label)
        legend_handles.append(bin_patch)
        legend_labels.append(bin_label)

    # Plot scatter plot with colored points based on Bin#
    if plt_log_transform is True:
        plt.scatter(range(len(sorted_working_df)), np.log10(sorted_working_df[read_count_column_name]+1), c=sorted_working_df['Bin'], cmap=cmap)
        plt.ylabel('Log(Read count+1)')
    else:
        plt.scatter(range(len(sorted_working_df)), sorted_working_df[read_count_column_name], c=sorted_working_df['Bin'], cmap=cmap)
        plt.ylabel('Read count')

    plt.xlabel('Rank of Activity [High→Low]')
    plt.title('Read count vs Rank\n(colored by Bin#)')

    # Add the legend on the left side of the plot
    plt.legend(handles=legend_handles, labels=legend_labels, loc='upper right')
    
    plt.show()
    

def plot_train_test_validation_folds(working_df, n_splits, read_count_column_name='Read count', plt_log_transform=True):
    """
    Plots the train, test, and validation indices for each Rotation of combined folds in a stratified K-fold cross-validation.

    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the fold labels and read counts.
        n_splits (int): Number of rotations in the cross-validation.
        read_count_column_name (str): Name of the column in the text file that contains the read count. Defaults to 'Read count'.
        plt_log_transform (bool):
            -False (default): Plots read count as in working_df
            -True: Plots a log 10 transform of read counts.

    Returns:
        None
    """

    # Create a list of fold columns based on the working DataFrame
    fold_cols = [f'Fold {i}' for i in range(working_df.filter(like='Fold').shape[1])]

    # Create subplots for each rotation
    fig, axes = plt.subplots(nrows=len(fold_cols), figsize=(8, 0.6*n_splits), sharex=True)
    fig.subplots_adjust(hspace=0.2, left=0.15)

    # Iterate over each fold column and plot the train, test, and validation indices
    for i, fold_col in enumerate(fold_cols):
        # Retrieve the train, test, and validation indices based on the fold column and plot type
        if plt_log_transform:
            train_indices = np.log10(working_df.loc[working_df[fold_col] == 'Train', read_count_column_name] + 1)
            test_indices = np.log10(working_df.loc[working_df[fold_col] == 'Test', read_count_column_name] + 1)
            val_indices = np.log10(working_df.loc[working_df[fold_col] == 'Validation', read_count_column_name] + 1)
        else:
            train_indices = working_df.loc[working_df[fold_col] == 'Train', read_count_column_name]
            test_indices = working_df.loc[working_df[fold_col] == 'Test', read_count_column_name]
            val_indices = working_df.loc[working_df[fold_col] == 'Validation', read_count_column_name]

        # Plot the train, test, and validation indices using scatter plots
        axes[i].scatter(train_indices, [0] * len(train_indices), label='Train', marker="|", color='black', s=20)
        axes[i].scatter(test_indices, [1] * len(test_indices), label='Test', marker="|", color='red', s=20)
        axes[i].scatter(val_indices, [2] * len(val_indices), label='Validation', marker="|", color='blue', s=20)

        # Set the y-axis limits and labels
        axes[i].set_ylim([-0.5, 2.5])
        axes[i].set_yticks([0, 1, 2])
        axes[i].set_yticklabels(['Train', 'Test', 'Validation'])
        axes[i].set_ylabel(f'Rotation {i}', rotation=0, ha='right', va='center')

    # Set the x-axis label based on the plot type
    axes[-1].set_xlabel('Log(Read count + 1)' if plt_log_transform else read_count_column_name)

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()

def plot_fold_subsets(working_df, n_splits, read_count_column_name='Read count', plt_log_transform=False):
    """
    Plots test indices for each fold, displaying the splits of the data

    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the fold labels and read counts.
        n_splits (int): Number of folds in the cross-validation.
        read_count_column_name (str): Name of the column in the text file that contains the read count. Defaults to 'Read count'.
        plt_log_transform (bool):
            -False (default): Plots read count as in working_df
            -True: Plots a log 10 transform of read counts.

    Returns:
        None
    """

    # Create a list of fold columns based on the working DataFrame
    fold_cols = [f'Fold {i}' for i in range(working_df.filter(like='Fold').shape[1])]

    # Create subplots for each fold
    fig, axes = plt.subplots(nrows=len(fold_cols), figsize=(4, 0.4*n_splits), sharex=True)
    fig.subplots_adjust(hspace=0.1, left=0.15)

    # Iterate over each fold column and plot the train, test, and validation indices
    for i, fold_col in enumerate(fold_cols):
        # Retrieve the train, test, and validation indices based on the fold column and plot type
        if plt_log_transform:
            test_indices = np.log10(working_df.loc[working_df[fold_col] == 'Test', read_count_column_name] + 1)
        else:
            test_indices = working_df.loc[working_df[fold_col] == 'Test', read_count_column_name]

        # Plot the train, test, and validation indices using scatter plots
        axes[i].scatter(test_indices, [1] * len(test_indices), label='Test', marker="|", color='crimson', s=50)

        # Set the y-axis limits and labels
        axes[i].set_ylim([-0.5, 2.5])
        axes[i].set_yticks([0, 1, 2])
        axes[i].set_yticklabels(['', '',''])
        axes[i].set_ylabel(fold_col, rotation=0, ha='right', va='center')

    # Set the x-axis label based on the plot type
    axes[-1].set_xlabel('Log(Read count + 1)' if plt_log_transform else read_count_column_name)

    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()
    return

In [23]:
def asymmetric_binning(data, read_count_column_name='Read count', bin_zeros_seperately=True, plt_log_transform=False, sep='\t'):
    """
    Apply asymmetric binning to read counts in a text file or DataFrame and assign bin numbers.
    
    Args:
        data (str or pandas.DataFrame): Path to the text file or a DataFrame containing the read counts.
        read_count_column_name (str): Name of the column that contains the read count. Defaults to 'Read count'.
        bin_zeros_seperately (bool): Determines if zero-scoring read counts is binned separately.
            - True (default): Zero-scoring read counts are binned into an exclusive bin.
            - False: Zero-scoring read counts are binned with the lowest bin.
        plt_log_transform (bool):
            - False (default): Plots read count as in working_df.
            - True: Plot a log 10 transform of read counts.
        sep (str): Separator used in the text file. Defaults to '\t'.
            
    Returns:
        working_df (pandas.DataFrame): DataFrame with additional bin numbers assigned.
        bin_ranges (pandas.DataFrame): DataFrame containing information about the bin ranges for the asymmetric binning strategy.
    """
    # Boundaries of the asymmetric bin widths (percent of maximum read count)
    bin_limits = ['0.0', '0.005', '0.01', '0.02', '0.04', '0.08', '0.15', '0.3', '1']
    
    # Check if data is a DataFrame or a file path
    if isinstance(data, pd.DataFrame):
        working_df = data.copy()
    else:
        with open(data) as file:
            # Read the contents into a Pandas DataFrame
            working_df = pd.read_csv(file, sep=sep)
        
    # Check if the read_count_column_name exists in the DataFrame
    if read_count_column_name not in working_df.columns:
        raise ValueError(f"The specified read_count_column_name '{read_count_column_name}' does not exist.\nThe DataFrame contains: {list(working_df.columns)}")

    # Convert read counts to float
    working_df[read_count_column_name] = working_df[read_count_column_name].astype(float)
    
    # Save the max read count for percent calculations
    max_read_count = working_df[read_count_column_name].max()

    # Create a DataFrame to store bin information
    bin_ranges = pd.DataFrame(columns=['Bin#', 'Range', 'Lower Read Count', 'Upper Read Count', 'Bin Read Count Range'])

    # Add a row for the zero-value bin
    if bin_zeros_seperately:
        bin_ranges.loc[0] = [0, '0.0', 0.0, 0.0, 'x=0']
    
    # Add rows for each bin determined by bin_limits
    for i in range(len(bin_limits) - 1):
        # Determine bin upper and lower boundaries
        lower_read_count = float(bin_limits[i]) * max_read_count
        upper_read_count = float(bin_limits[i + 1]) * max_read_count
        
        # Create bin-specific information and fill DataFrame
        bin_range_str = f"{bin_limits[i]}-{bin_limits[i + 1]}"
        if lower_read_count == 0.0 and not bin_zeros_seperately:
            bin_read_range_str = f"{lower_read_count:.2f}≤x≥{upper_read_count:.2f}"
        else:
            bin_read_range_str = f"{lower_read_count:.2f}<x≥{upper_read_count:.2f}"
        bin_ranges.loc[i + 1] = [i + 1, bin_range_str, lower_read_count, upper_read_count, bin_read_range_str]

    # Add bin column to DataFrame
    working_df['Bin'] = 0

    # Assign each read count to a bin
    for i, row in bin_ranges.iterrows():
        lower_range = row['Lower Read Count']
        upper_range = row['Upper Read Count']
        
        # Assign bin number to appropriate values 
        if lower_range == 0.0 and not bin_zeros_seperately:
            mask = (working_df[read_count_column_name] >= lower_range) & (working_df[read_count_column_name] <= upper_range)
        else:
            mask = (working_df[read_count_column_name] > lower_range) & (working_df[read_count_column_name] <= upper_range)
        working_df.loc[mask, 'Bin'] = i

    # Count the number of items in each bin and print the result
    bin_counts = working_df.groupby('Bin').size()
    print(bin_counts)
    
    # Create plot of bins and read count 
    plot_bins_read_count_by_rank(working_df, bin_ranges, read_count_column_name=read_count_column_name, plt_log_transform=plt_log_transform)
    
    return working_df, bin_ranges

def create_stratified_folds(working_df, n_splits=20, read_count_column_name='Read count', bin_column_name='Bin', plt_log_transform=False):
    """
    Performs stratified K-fold cross-validation, assigns fold labels to the working DataFrame,
    plots the train, test, and validation indices for each fold, and visualizes two-sample kolmogorov-smirnov test statistics and p-values.

    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the data for cross-validation.
        n_splits (int): Number of folds in the cross-validation. Defaults to 20.
        read_count_column_name (str): Name of the column in the text file that contains the read count. Defaults to 'Read count'.
        bin_column_name (str): Name of the column in the DataFrame that contains the bin assignments. Defaults to 'Bin'.
        plt_log_transform (bool):
            -False (default): Plots read count as in working_df
            -True: Plots a log 10 transform of read counts.
    Returns:
        pandas.DataFrame: Modified working DataFrame with fold labels assigned.

    """

    # Establish stratified K-fold parameters
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7)
    
    # Pull bin assignments
    bins = working_df[bin_column_name]
    
    # Perform stratified K-fold cross-validation and store fold indices
    fold_indices = []
    for _, infold_index in skf.split(bins, bins):
        fold_indices.append(infold_index.tolist())
    
    # Iterate over the fold indices and assign fold labels to the working DataFrame
    for fold_no in range(len(fold_indices)):
        fold_label = f'Fold {fold_no}'
        working_df[fold_label] = '0'

        # Determine the validation, test, and train indices for the current fold
        val_index = [fold_indices[(fold_no - 1) % len(fold_indices)]]
        val_index = list(itertools.chain.from_iterable(val_index))

        test_index = [fold_indices[(fold_no - 2) % len(fold_indices)]]
        test_index = list(itertools.chain.from_iterable(test_index))

        train_index = [i % len(fold_indices) for i in range(fold_no, fold_no - 2 + len(fold_indices))]
        train_index = [fold_indices[i] for i in train_index]
        train_index = list(itertools.chain.from_iterable(train_index))

        # Assign fold labels to the corresponding indices in the working DataFrame
        working_df.loc[val_index, fold_label] = 'Validation'
        working_df.loc[test_index, fold_label] = 'Test'
        working_df.loc[train_index, fold_label] = 'Train'

    # Plot train, test, and validation indices for each fold
    plot_train_test_validation_folds(working_df, n_splits, read_count_column_name, plt_log_transform)

    # Sort the fold columns in ascending order
    sorted_cols = sorted(working_df.filter(regex='Fold \d+$').columns, key=lambda x: int(x.split()[-1]))

    statistic_values = []
    pvalue_values = []

    # Calculate KS test statistics and p-values for each fold
    for fold in sorted_cols:
        #KS-test between testing to the whole dataset
        stat_test2all =  ks_2samp(working_df.loc[working_df[fold] == 'Test', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).statistic
        pval_test2all = ks_2samp(working_df.loc[working_df[fold] == 'Test', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).pvalue

        #KS-test between validation to the whole dataset
        stat_val2all = ks_2samp(working_df.loc[working_df[fold] == 'Validation', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).statistic
        pval_val2all = ks_2samp(working_df.loc[working_df[fold] == 'Validation', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).pvalue
        
        #KS-test between training to the whole dataset
        stat_train2all = ks_2samp(working_df.loc[working_df[fold] == 'Train', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).statistic
        pval_train2all = ks_2samp(working_df.loc[working_df[fold] == 'Train', read_count_column_name], working_df.loc[working_df[fold].isin(['Train', 'Validation', 'Test']), read_count_column_name]).pvalue

        #KS-test between validation to the test fold
        stat_val2test = ks_2samp(working_df.loc[working_df[fold] == 'Validation', read_count_column_name], working_df.loc[working_df[fold] == 'Test', read_count_column_name]).statistic
        pval_val2test = ks_2samp(working_df.loc[working_df[fold] == 'Validation', read_count_column_name], working_df.loc[working_df[fold] == 'Test', read_count_column_name]).pvalue
        
        #collect the stats for each rotation
        statistic_values.append([stat_test2all, stat_val2all, stat_train2all, ])
        pvalue_values.append([pval_test2all, pval_val2all, pval_train2all,])

    # Plot the KS test statistic and p-value values
    statistic_values = np.array(statistic_values)
    pvalue_values = np.array(pvalue_values)

    # Create a figure with two subplots side by side
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Box plot for KS test statistic values
    axs[0].set_title('KS Test Statistic')
    axs[0].set_xlabel('Compared to Whole Dataset')
    axs[0].set_ylabel('KS Statistic Value')
    axs[0].set_ylim(0,0.1)
    
    # Using boxplot function to create the boxplot
    bp = axs[0].boxplot(statistic_values, labels=['Test', 'Validation', 'Train'], patch_artist=True)

    # Set colors for boxplot elements
    colors = ['red', 'blue', 'black']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)


    # Box plot for p-values
    axs[1].set_title('KS Test P-Value')
    axs[1].set_xlabel('Compared to Whole Dataset')
    axs[1].set_ylabel('P-Value')
    axs[1].set_ylim(0.5,1.05)
    
    
    # Using boxplot function to create the boxplot for p-values
    bp_pvalue = axs[1].boxplot(pvalue_values, labels=['Test', 'Validation', 'Train'], patch_artist=True)

    # Set colors for boxplot elements
    for patch, color in zip(bp_pvalue['boxes'], colors):
        patch.set_facecolor(color)

    plt.show()

    return working_df


def encode_sequences_to_numpy_array(working_df, sequence_column_name='sequence'):
    """
    Encode sequences from a text file into a numpy array of one-hot encoded sequences.

    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the sequences.
        sequence_column_name (str): Name of the column in the text file that contains the sequences. Defaults to 'sequence'.

    Returns:
        one_hot_seq (numpy.ndarray): A numpy array containing the one-hot encoded sequences.
    """
    # Define a mapping from nucleotides to integers
    nt_to_int = {'A': 0, 'T': 1, 'G': 2, 'C': 3, 'a': 0, 't': 1, 'g': 2, 'c': 3}
        
    # Check if the sequence_column_name exists in the DataFrame
    if sequence_column_name not in working_df.columns:
        raise ValueError(f"The specified sequence_column_name '{sequence_column_name}' does not exist in the file.\nThe file contains: '{list(working_df.columns)}'")

    # Define a new column to store the one-hot encoded sequences
    working_df['one_hot_seq'] = ''

    # Loop through each row in the DataFrame
    for index, row in working_df.iterrows():
        # Convert the sequence to an integer representation
        int_seq = [nt_to_int[nt] for nt in row[sequence_column_name]]

        # One-hot encode the integer sequence
        one_hot_seq = tf.keras.utils.to_categorical(int_seq, num_classes=4)

        # Update the 'one_hot_seq' column with the one-hot encoded sequence
        working_df.at[index, 'one_hot_seq'] = one_hot_seq

    # Convert the 'one_hot_seq' column to a numpy array
    one_hot_seq = np.array(list(working_df['one_hot_seq']))
    
    # Flatten each array in ins
    one_hot_seq = np.reshape(one_hot_seq, (one_hot_seq.shape[0], -1))
    
    # Generate feature names based on sequence length
    sequence_length = len(working_df[sequence_column_name].iloc[0])  # Assuming all sequences are of the same length
    feature_names = [f"Position_{i+1}_{base}" for i in range(sequence_length) for base in ['A', 'T', 'G', 'C']]

    return one_hot_seq, feature_names


def normalize_read_count_to_numpy_array(working_df, read_count_column_name='Read count', min_max_log=False):
    """
    Normalize read counts from a pandas DataFrame and return them as a numpy array.

    Args:
        working_df (pandas.DataFrame): Input DataFrame containing the Read counts.
        read_count_column_name (str): Name of the column in the text file that contains the Read Count. Defaults to 'Read count'.
        min_max_log (bool): Determines the type of normalization to be applied.
            - False (default): Normalize using min-max normalization.
            - True: Normalize using min-max normalization after applying a logarithmic transformation (log base 10) to the read counts.
        
    Returns:
        normalized_read_counts (numpy.ndarray): A numpy array containing the normalized Read counts.
    """
    # Check if the read_count_column_name exists in the DataFrame
    if read_count_column_name not in working_df.columns:
        raise ValueError(f"The specified read_count_column_name '{read_count_column_name}' does not exist in the file.\nThe file contains: '{list(working_df.columns)}'")
        
    # Assign read count to variable
    read_counts = working_df[read_count_column_name].astype(float)

    if min_max_log == True:
        # Normalize read counts with a logarithmic transformation (log base 10)
        read_counts = np.log(read_counts + 1) # Add 1 to accommodate log of zero values

    # Normalize read counts using min-max normalization
    minimum = np.amin(read_counts)
    maximum = np.amax(read_counts)
    normalized_read_counts = (read_counts-minimum)/(maximum-minimum)
    normalized_read_counts= normalized_read_counts.to_numpy()

    return normalized_read_counts


def model_training_crossfold_validation(model_building_function, working_df, sequence_variables, read_count_labels):
    """
    Perform crossfold validation for model training and evaluation.
    
    Plots loss over epochs, model prediction performance for each fold, and tfa r2_score (FVAF) over epochs

    Args:
        model_building_function (function): A function that builds the model architecture.
        working_df (pd.DataFrame): The working dataframe containing the fold assignments.
        sequence_variables (np.ndarray): The sequence variables.
        read_count_labels (np.ndarray): The read count labels.

    Returns:
        pd.DataFrame: A DataFrame with columns for loss metric values, training performance values, and validation performance values.        

    """
    # Suppress TensorFlow info messages to prevent repeated model summary printing
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    
    X = sequence_variables
    y = read_count_labels
    
    # Initialize empty list to cache fold's performance metric
    metric_values = []
    loss_metric_values = []
    training_performance_values = []
    validation_performance_values = []
    spearman_train_values = []
    spearman_val_values = []
    rmse_train_values = []
    rmse_val_values = []

    # Pull n_Fold columns to list
    fold_columns = [col for col in working_df.columns if col.startswith('Fold')]

    num_folds = len(fold_columns)
    num_rows = num_folds  # Number of rows in the grid
    num_cols = 3  # Number of columns in the grid (loss plot and validation plot)

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, num_rows*3))

    for i, n_fold in enumerate(fold_columns):
        train_indices = working_df.index[working_df[n_fold] == 'Train'].tolist()  # Indices for train label
        test_indices = working_df.index[working_df[n_fold] == 'Test'].tolist()  # Indices for test label
        validation_indices = working_df.index[working_df[n_fold] == 'Validation'].tolist()  # Indices for validation label
    
        # Print the fold label
        print(f"Fold: {n_fold}")
        
        # Connect train_indices to X_train and y_train
        X_train = X[train_indices]
        y_train = y[train_indices]
        
        # Connect validation_indices to X_val and y_val
        X_val = X[validation_indices]
        y_val = y[validation_indices]

        # Early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Build model architecture using the provided function
        working_model = model_building_function(sequence_variables)
        
        # Train the model on the current training set
        working_history = working_model.fit(X_train, y_train, epochs=100, verbose=0, validation_data=(X_val, y_val), callbacks=[early_stop])
        
        # Predict the values for the training and validation sets
        y_pred_train = working_model.predict(X_train).flatten()
        y_pred_val = working_model.predict(X_val).flatten()

        # Compute the accuracy, loss, or any other metric of interest
        loss_metric_value = working_model.evaluate(X_val, y_val) #MSE
        training_performance_value = FVAF(y_train, y_pred_train) #training FVAF
        validation_performance_value = FVAF(y_val, y_pred_val) #validation FVAF
        spearman_train, _ = spearmanr(y_train, y_pred_train) #training spearman
        spearman_val, _ = spearmanr(y_val, y_pred_val) #validation spearman
        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train)) #training RMSE
        rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val)) #validation RMSE
        
    
        # Get the R2 scores from the history object (using the FVAF version of R2)
        r2_scores_train = working_history.history['r_square']
        r2_scores_val = working_history.history['val_r_square']

        # Plot the loss curve in the left subplot
        axs[i, 0].plot(working_history.history['loss'], color = 'black')
        axs[i, 0].plot(working_history.history['val_loss'], color = (0.2667, 0.5020, 0.9490))
        axs[i, 0].set_xlabel('Epoch')
        axs[i, 0].set_ylabel('Loss')
        #axs[i, 0].set_title('{}'.format(n_fold))
        # Replace "fold" with "rotation" in the title
        title_text = str(n_fold).replace('Fold', 'Rotation')
        axs[i, 0].set_title(title_text)
        
        
        axs[i, 0].legend(['Train', 'Validation'], loc='upper right')

        # Plot the scatter plot of real vs predicted values in the right subplot
        axs[i, 1].scatter(y_train, y_pred_train, s=30, color = 'black', label='Training: {:.3f}'.format(training_performance_value)+' FVAF')
        axs[i, 1].scatter(y_val, y_pred_val, s=30, color = (0.2667, 0.5020, 0.9490), label='Validating: {:.3f}'.format(validation_performance_value)+' FVAF')
        axs[i, 1].set_title('Real vs Predicted Values')
        axs[i, 1].set_xlabel('Real Values')
        axs[i, 1].set_ylabel('Predicted Values')
        axs[i, 1].set_xlim([0, np.max(y)])
        axs[i, 1].set_ylim([0, np.max(y)])
        axs[i, 1].set_aspect('equal')
        axs[i, 1].legend(loc='upper left', fontsize='small')
              
        # Plot the R2 curve in the 3rd subplot
        axs[i, 2].plot(r2_scores_train, color='black')
        axs[i, 2].plot(r2_scores_val, color=(0.8667, 0.3020, 0.2941))
        axs[i, 2].set_title('Performance per Epoch')
        axs[i, 2].set_ylim([-1.1,1.1])
        axs[i, 2].set_xlabel('Epoch')
        axs[i, 2].set_ylabel('FVAF Score')
        axs[i, 2].legend(['Train', 'Validation'], loc='lower right')

        # Append the metric value and indices to the cache lists
        loss_metric_values.append(loss_metric_value[0])
        training_performance_values.append(training_performance_value)
        validation_performance_values.append(validation_performance_value)
        spearman_train_values.append(spearman_train)
        spearman_val_values.append(spearman_val)
        rmse_train_values.append(rmse_train)
        rmse_val_values.append(rmse_val)

        # Print the metric value for this fold
        print('Fold loss metric value: {}'.format(loss_metric_value)+' Mean Square Error')
        print('Fold training performance: {}'.format(training_performance_value)+' FVAF')
        print('Fold validation performance: {}'.format(validation_performance_value)+' FVAF')
    
    print(working_model.summary())
    
    plt.tight_layout()
    plt.show()
    
    metric_df = pd.DataFrame(
        {
            'Validation_MSE': loss_metric_values,
            'Training_FVAF': training_performance_values,
            'Validation_FVAF': validation_performance_values,
            'Training_Spearman': spearman_train_values,
            'Validation_Spearman': spearman_val_values,
            'Training_RMSE': rmse_train_values,
            'Validation_RMSE': rmse_val_values
        },
        index=fold_columns)
    
    return metric_df

# █████████████████████████████████
# H4S2 Model  Hyperparameter Search
## Repeated through multiple Hyperparameter values
# █████████████████████████████████

## • Dropout Hyperparameter Search 

In [24]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')

hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = [0, 0.2, 0.4, 0.6, 0.65, 0.7, 0.8]
trialing_hyperparameter_label = 'dropout'

for i in range(len(trialing_hyperparameter_values)):
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = trialing_hyperparameter_values[i]

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            tf.keras.layers.Dense(1, activation='exponential')
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

C:\Users\jchar\AppData\Local\Temp\ipykernel_28896\3053369207.py:2: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('Agg')
C:\Users\jchar\AppData\Local\Temp\ipykernel_28896\532212000.py:92: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Bin
0    1778
1    1054
2     120
3     299
4     350
5     233
6     146
7      81
8      35
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_28896\532212000.py:146: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_28896\854057990.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_28896\532212000.py:193: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0


KeyboardInterrupt: 

## •Learning Rate Hyperparameter Search 

In [43]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')

hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = [0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.005, 0.01]
trialing_hyperparameter_label = 'lrate'

for i in range(len(trialing_hyperparameter_values)):
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = 0.65

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            tf.keras.layers.Dense(1, activation='exponential')
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=trialing_hyperparameter_values[i])

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 0.0018 - r_square: 0.0423
Fold loss metric value: [0.0017733228160068393, 0.042335450649261475] Mean Square Error
Fold training performance: 0.02554142689289829 FVAF
Fold validation performance: 0.04233563139950136 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 8.1039e-04 - r_square: 0.7352
Fold loss metric value: [0.0008103854488581419, 0.7352315187454224] Mean Square Error
Fold training performance: 0.8255209791539634 FVAF
Fold validation performance: 0.7352315134245092 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 0.0033 - r_square: -4.9448e-04
Fold loss metric value: [0.0033345723059028387, -0.0004944801330566406] Mean Square Error
Fold training performance: -0.004398905791091368 FVAF
Fold validation performance: -0.0004946204764739726 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 0.0059 - r_square: 0.0042
Fold loss metr

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_274 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_274 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


...

C:\Users\jchar\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


7/7 [==============================] - 0s 2ms/step - loss: 0.0031 - r_square: -0.1307
Fold loss metric value: [0.003068057121708989, -0.13069689273834229] Mean Square Error
Fold training performance: -0.10360577609113775 FVAF
Fold validation performance: -0.13069693973338103 FVAF
Model: "sequential_394"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_394 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_394 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Tra

C:\Users\jchar\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


         Validation_MSE_lrate_5e-05  Training_FVAF_lrate_5e-05  \
Fold 0                     0.001773                   0.025541   
Fold 1                     0.000810                   0.825521   
Fold 2                     0.003335                  -0.004399   
Fold 3                     0.005921                  -0.004825   
Fold 4                     0.004357                   0.011679   
Fold 5                     0.000929                   0.791499   
Fold 6                     0.002258                  -0.010219   
Fold 7                     0.003494                  -0.001707   
Fold 8                     0.006312                   0.039635   
Fold 9                     0.004388                   0.007481   
Fold 10                    0.004572                   0.019203   
Fold 11                    0.005336                   0.010691   
Fold 12                    0.003557                   0.029196   
Fold 13                    0.000376                   0.808991   
Fold 14   

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2879106992.py:116: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('module://ipykernel.pylab.backend_inline')


## •Hidden Activation Function Hyperparameter Search 

In [47]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')


hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = ['leaky_relu','relu','elu','linear','sigmoid']
trialing_hyperparameter_label = 'hidden_activation_function'

for i in range(len(trialing_hyperparameter_values)):
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = 0.65

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation=trialing_hyperparameter_values[i]),
            #tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation=trialing_hyperparameter_values[i]),
            #tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            
            tf.keras.layers.Dense(1, activation='exponential')
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\1011450942.py:2: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('Agg')
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 6.3781e-04 - r_square: 0.6556
Fold loss metric value: [0.0006378146354109049, 0.6555548310279846] Mean Square Error
Fold training performance: 0.8123665391206518 FVAF
Fold validation performance: 0.6555548925007293 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 0.0031 - r_square: -0.0036
Fold loss metric value: [0.003071680199354887, -0.0035767555236816406] Mean Square Error
Fold training performance: -0.004662200046227083 FVAF
Fold validation performance: -0.0035768900485153576 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 9.4381e-04 - r_square: 0.7168
Fold loss metric value: [0.0009438149281777442, 0.7168207764625549] Mean Square Error
Fold training performance: 0.829275120159485 FVAF
Fold validation performance: 0.7168207474187358 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 0.0059 - r_square: 0.0053
Fold loss metric val

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_716 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_716 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


...

Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:322: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, num_rows*3))


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 0.0018 - r_square: 0.0022 
Fold loss metric value: [0.0018475813558325171, 0.002232849597930908] Mean Square Error
Fold training performance: -0.00028502945276942526 FVAF
Fold validation performance: 0.0022329302783659077 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 7.8037e-04 - r_square: 0.7450
Fold loss metric value: [0.0007803667685948312, 0.7450392246246338] Mean Square Error
Fold training performance: 0.789120998210623 FVAF
Fold validation performance: 0.745039193657812 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 0.0033 - r_square: 0.0041
Fold loss metric value: [0.003319187555462122, 0.004121541976928711] Mean Square Error
Fold training performance: 0.004280945643443723 FVAF
Fold validation performance: 0.0041214132529699965 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 0.0059 - r_square: 0.0019
Fold loss metric va

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_796 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_796 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
         Validation_MSE_hidden_activation_function_leaky_relu  \
Fold 0                                            0.000638      
Fold 1                                            0.003072      
Fold 2                                            0.000944      
Fold 3                               

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\1011450942.py:120: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('module://ipykernel.pylab.backend_inline')


## •Output Activation Function Hyperparameter Search 

In [48]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')


hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = ['leaky_relu','relu','elu','linear','sigmoid','exponential']

trialing_hyperparameter_label = 'output_activation_function'

for i in range(len(trialing_hyperparameter_values)):
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = 0.65

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            
            #tf.keras.layers.Dense(1, activation='exponential')
            tf.keras.layers.Dense(1, activation=trialing_hyperparameter_values[i])
            
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 6.0203e-04 - r_square: 0.6749
Fold loss metric value: [0.0006020326400175691, 0.6748784780502319] Mean Square Error
Fold training performance: 0.8239483298994947 FVAF
Fold validation performance: 0.6748785406479549 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 6.0735e-04 - r_square: 0.8016
Fold loss metric value: [0.000607354799285531, 0.8015655279159546] Mean Square Error
Fold training performance: 0.8482912663608019 FVAF
Fold validation performance: 0.8015655371352742 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 8.0288e-04 - r_square: 0.7591
Fold loss metric value: [0.0008028803858906031, 0.7591063380241394] Mean Square Error
Fold training performance: 0.877770494430043 FVAF
Fold validation performance: 0.7591062792993339 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 9.7378e-04 - r_square: 0.8362
Fold loss metric value: 

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_816 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_816 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


...

Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 8.4413e-04 - r_square: 0.5441
Fold loss metric value: [0.0008441315731033683, 0.5441355109214783] Mean Square Error
Fold training performance: 0.8060986367382319 FVAF
Fold validation performance: 0.544135562122258 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 6.8771e-04 - r_square: 0.7753
Fold loss metric value: [0.0006877092528156936, 0.7753122448921204] Mean Square Error
Fold training performance: 0.8383000636129568 FVAF
Fold validation performance: 0.7753121706313237 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 6.8882e-04 - r_square: 0.7933
Fold loss metric value: [0.0006888228817842901, 0.7933277487754822] Mean Square Error
Fold training performance: 0.8235364987114095 FVAF
Fold validation performance: 0.7933277308912683 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 7.0875e-04 - r_square: 0.8808
Fold loss metric value:

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_916 (Dropout)       (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_916 (Dense)           (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
         Validation_MSE_output_activation_function_leaky_relu  \
Fold 0                                            0.000602      
Fold 1                                            0.000607      
Fold 2                                            0.000803      
Fold 3                               

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\493766636.py:121: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('module://ipykernel.pylab.backend_inline')


## •Hidden Layer Architecture Hyperparameter Search

In [49]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')


hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = [[50,25], [100,50], [200,100], [300,150], [400,200],
                                  [25,25], [50,50], [100,100], [200,200], [400,400],
                                  [25,50], [50,100], [100,200], [150,300], [200,400]
                                 ]
trialing_hyperparameter_label = 'hidden_layer_architecture'

for i in range(len(trialing_hyperparameter_values)):
    print(trialing_hyperparameter_values[i])
    print(trialing_hyperparameter_values[i][0])
    
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = 0.65

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            #tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            tf.keras.layers.Dense(trialing_hyperparameter_values[i][0], use_bias=True, name="hidden_1", activation='relu'),
            
            tf.keras.layers.Dropout(dropout_rate),
            #tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            tf.keras.layers.Dense(trialing_hyperparameter_values[i][1], use_bias=True, name="hidden_2", activation='relu'),
            
            tf.keras.layers.Dense(1, activation='exponential')
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[50, 25]
50
Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 0.0019 - r_square: -0.0136
Fold loss metric value: [0.0018769692396745086, -0.013637781143188477] Mean Square Error
Fold training performance: -0.004349414475058211 FVAF
Fold validation performance: -0.013637715455592891 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 0.0031 - r_square: -0.0180
Fold loss metric value: [0.00311576877720654, -0.017981290817260742] Mean Square Error
Fold training performance: -0.019038858899293354 FVAF
Fold validation performance: -0.017981515498802336 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 0.0034 - r_square: -0.0055
Fold loss metric value: [0.0033512571826577187, -0.0055005550384521484] Mean Square Error
Fold training performance: -0.005166015146321534 FVAF
Fold validation performance: -0.0055007257918917585 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 0.0059 - r_square: 0.0081
Fold los

 hidden_1 (Dense)            (None, 50)                5650      
                                                                 
 dropout_936 (Dropout)       (None, 50)                0         
                                                                 
 hidden_2 (Dense)            (None, 25)                1275      
                                                                 
 dense_936 (Dense)           (None, 1)                 26        
                                                                 
Total params: 6951 (27.15 KB)
Trainable params: 6951 (27.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


...

[200, 400]
200
Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 3ms/step - loss: 7.1504e-04 - r_square: 0.6138
Fold loss metric value: [0.0007150426390580833, 0.6138486266136169] Mean Square Error
Fold training performance: 0.7528237062258087 FVAF
Fold validation performance: 0.6138486596211616 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 7.4310e-04 - r_square: 0.7572
Fold loss metric value: [0.000743096461519599, 0.7572161555290222] Mean Square Error
Fold training performance: 0.8168464621940857 FVAF
Fold validation performance: 0.7572161138854077 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 7.5715e-04 - r_square: 0.7728
Fold loss metric value: [0.0007571532041765749, 0.7728261351585388] Mean Square Error
Fold training performance: 0.8167195039576054 FVAF
Fold validation performance: 0.7728261489595769 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 5.9185e-04 - r_square: 0.9005
Fold loss metric value:

                                                                 
 dropout_1216 (Dropout)      (None, 200)               0         
                                                                 
 hidden_2 (Dense)            (None, 400)               80400     
                                                                 
 dense_1216 (Dense)          (None, 1)                 401       
                                                                 
Total params: 103401 (403.91 KB)
Trainable params: 103401 (403.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
         Validation_MSE_hidden_layer_architecture_[50, 25]  \
Fold 0                                            0.001877   
Fold 1                                            0.003116   
Fold 2                                            0.003351   
Fold 3                                            0.005898   
Fold 4                                            0

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\4056555389.py:126: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('module://ipykernel.pylab.backend_inline')


## •Hidden Layer Bias use Hyperparameter Search

In [50]:
# Switch to the 'Agg' backend to suppress plots
plt.switch_backend('Agg')


hyperparameter_performance = pd.DataFrame()

trialing_hyperparameter_values = [True, False]
trialing_hyperparameter_label = 'hidden_bias'

for i in range(len(trialing_hyperparameter_values)):
    """
    Template to N Fold Cross Validate defined model.

    Workflow:
        Preprocess recombination data.
            - asymmetric_binning (function)
                └ plot_bins_read_count_by_rankc (function)
            - create_stratified_folds (function)
                └ plot_train_test_validation_folds (function)

        Normalize/encode labels and variables.
            - encode_sequences_to_numpy_array (function)
            - normalize_read_count_to_numpy_array (function)

        Establish model architecture and parameters.
            - build_model (function)

        Perform N Fold Cross Validation.
            - model_training_crossfold_validation (function)
                └ FVAF (function)

    """

    bin_assigned_df, table_of_bin_range_boundaries = asymmetric_binning(
        data=result_dataframes['iseq1'],
        read_count_column_name='Read count',
        bin_zeros_seperately=True,
        plt_log_transform=False,
        sep='\t')

    fold_assigned_df = create_stratified_folds(
        working_df=bin_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        bin_column_name='Bin',
        plt_log_transform=False)

    plot_fold_subsets(
        working_df=fold_assigned_df,
        n_splits=20,
        read_count_column_name='Read count',
        plt_log_transform=False)

    sequence_variables, sequence_labels = encode_sequences_to_numpy_array(
        working_df=fold_assigned_df,
        sequence_column_name='Sequence')
    read_count_labels = normalize_read_count_to_numpy_array(
        working_df=fold_assigned_df,
        read_count_column_name='Read count',
        min_max_log=False)

    def build_model(input_variables):
        """
        Construct Sequential NN model; called by the model_training_crossfold_validation function.

        Args:
            input_variables (np.ndarray): The sequence variables used to define layer shapes.

        Returns:
            tf.keras.Sequential: The constructed neural network model.
        """

        # Dropout variable
        dropout_rate = 0.65

        # Define the neural network architecture
        working_model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_variables[0].shape),
            #tf.keras.layers.Dense(300, use_bias=True, name="hidden_1", activation='relu'),
            tf.keras.layers.Dense(300, use_bias=trialing_hyperparameter_values[i], name="hidden_1", activation='relu'),
            
            tf.keras.layers.Dropout(dropout_rate),
            #tf.keras.layers.Dense(150, use_bias=True, name="hidden_2", activation='relu'),
            tf.keras.layers.Dense(150, use_bias=trialing_hyperparameter_values[i], name="hidden_2", activation='relu'),
            
            tf.keras.layers.Dense(1, activation='exponential')
        ])

        # Define the optimizer with a learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

        # Compile the working_model with the mean squared error loss function and the optimizer
        working_model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[tfa.metrics.r_square.RSquare()])

        return working_model

    cross_fold_performance_metrics = model_training_crossfold_validation(
        model_building_function=build_model,
        working_df=fold_assigned_df,
        sequence_variables=sequence_variables,
        read_count_labels=read_count_labels)

    # Add the dropout rate to the performance metrics
    cross_fold_performance_metrics.columns = [f"{col}_{trialing_hyperparameter_label}_{trialing_hyperparameter_values[i]}" for col in cross_fold_performance_metrics.columns]

    # Append the performance metrics to the DataFrame
    hyperparameter_performance = pd.concat([hyperparameter_performance, cross_fold_performance_metrics], axis=1)

# Display the combined performance metrics DataFrame
print(hyperparameter_performance)

# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}"

# Switch back to the default backend to enable interactive mode
plt.switch_backend('module://ipykernel.pylab.backend_inline')

Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 7.8355e-04 - r_square: 0.5769
Fold loss metric value: [0.0007835512515157461, 0.5768512487411499] Mean Square Error
Fold training performance: 0.7951643118377401 FVAF
Fold validation performance: 0.5768512980647791 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 7.9812e-04 - r_square: 0.7392
Fold loss metric value: [0.0007981245871633291, 0.7392374277114868] Mean Square Error
Fold training performance: 0.8187027390885523 FVAF
Fold validation performance: 0.7392373981535503 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 9.5291e-04 - r_square: 0.7141
Fold loss metric value: [0.0009529124945402145, 0.7140911817550659] Mean Square Error
Fold training performance: 0.8077966628370481 FVAF
Fold validation performance: 0.7140911473880491 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 5.2977e-04 - r_square: 0.9109
Fold loss metric value

 hidden_1 (Dense)            (None, 300)               33900     
                                                                 
 dropout_1236 (Dropout)      (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45150     
                                                                 
 dense_1236 (Dense)          (None, 1)                 151       
                                                                 
Total params: 79201 (309.38 KB)
Trainable params: 79201 (309.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:82: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Bin
0    1491
1    1335
2     123
3     287
4     355
5     235
6     146
7      88
8      36
dtype: int64


C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:136: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:203: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\3920272133.py:183: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Fold: Fold 0
7/7 [==============================] - 0s 2ms/step - loss: 8.2192e-04 - r_square: 0.5561
Fold loss metric value: [0.0008219185401685536, 0.5561314225196838] Mean Square Error
Fold training performance: 0.8049926941437591 FVAF
Fold validation performance: 0.556131502821255 FVAF
Fold: Fold 1
7/7 [==============================] - 0s 2ms/step - loss: 6.7605e-04 - r_square: 0.7791
Fold loss metric value: [0.0006760470569133759, 0.7791224718093872] Mean Square Error
Fold training performance: 0.8410109021904426 FVAF
Fold validation performance: 0.7791224483927298 FVAF
Fold: Fold 2
7/7 [==============================] - 0s 2ms/step - loss: 7.7044e-04 - r_square: 0.7688
Fold loss metric value: [0.0007704411982558668, 0.7688392400741577] Mean Square Error
Fold training performance: 0.8261175720535807 FVAF
Fold validation performance: 0.7688392225747458 FVAF
Fold: Fold 3
7/7 [==============================] - 0s 2ms/step - loss: 5.5915e-04 - r_square: 0.9060
Fold loss metric value:

 hidden_1 (Dense)            (None, 300)               33600     
                                                                 
 dropout_1256 (Dropout)      (None, 300)               0         
                                                                 
 hidden_2 (Dense)            (None, 150)               45000     
                                                                 
 dense_1256 (Dense)          (None, 1)                 151       
                                                                 
Total params: 78751 (307.62 KB)
Trainable params: 78751 (307.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
         Validation_MSE_hidden_bias_True  Training_FVAF_hidden_bias_True  \
Fold 0                          0.000784                        0.795164   
Fold 1                          0.000798                        0.818703   
Fold 2                          0.000953                        0.807

C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2601357644.py:412: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\jchar\AppData\Local\Temp\ipykernel_39596\2188673881.py:120: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  plt.switch_backend('module://ipykernel.pylab.backend_inline')


In [ ]:
# Generate an Excel file with the combined performance metrics
output_file = f"hyperparameter_search_{trialing_hyperparameter_label}.xlsx"
hyperparameter_performance.to_excel(output_file, index=False)

print(f"Results have been saved to {output_file}")